# Chroma key implementation

In [12]:
import numpy as np
import cv2 as cv
import time

In [17]:
img_size = (480, 270)
lower_bound = np.array([0, 180, 0])
upper_bound = np.array([130,255,130])
video_path = '../data/chroma.mp4'
replacement_path = '../data/map.jpg'

In [18]:
def process_video(video_path, replacement_path, img_size, lower_bound, upper_bound, process_func):
    start = time.time()
    is_chroma = 0
    video = cv.VideoCapture(video_path)
    replacement = cv.resize(cv.imread(replacement_path), img_size)
    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break

        key = cv.waitKey(10)
        if key == ord('q'):
            break
        elif key != -1:
            is_chroma = (is_chroma + 1) % 2

        frame = cv.resize(frame, img_size)

        if is_chroma:
            frame = process_func(frame, replacement, lower_bound, upper_bound)

        cv.imshow('frame', frame)
    video.release()
    cv.destroyAllWindows()
    end = time.time()
    return end-start

### Opencv builtin function

In [19]:
def chroma_opencv(frame, replacement, lower_bound, upper_bound):
    mask = cv.inRange(frame, lower_bound, upper_bound)
    res = cv.bitwise_and(frame, frame, mask = mask)
    frame = frame - res
    frame = np.where(frame != 0, frame, replacement)
    return frame

In [21]:
time_spent = process_video(video_path, replacement_path, img_size, lower_bound, upper_bound, chroma_opencv)
print(round(time_spent, 2))

19.97


### Only python implementation

In [22]:
def chroma_python(frame, replacement, lower_bound, upper_bound):
    if frame.shape != replacement.shape:
        return frame

    for i in range(frame.shape[0]):
        for j in range(frame.shape[1]):
            pixel = frame[i][j]
            if lower_bound[0] <= pixel[0] <= upper_bound[0] and lower_bound[1] <= pixel[1] <= upper_bound[1] and lower_bound[2] <= pixel[2] <= upper_bound[2]:
                frame[i][j] = replacement[i][j]
    return frame

In [23]:
time_spent = process_video(video_path, replacement_path, img_size, lower_bound, upper_bound, chroma_python)
print(round(time_spent, 2))

528.42


### JIT implementation

In [24]:
from numba import jit

In [35]:
@jit
def chroma_jit(frame, replacement, lower_bound, upper_bound):
    if frame.shape != replacement.shape:
        return frame

    for i in range(frame.shape[0]):
        for j in range(frame.shape[1]):
            pixel = frame[i][j]
            if lower_bound[0] <= pixel[0] <= upper_bound[0] and lower_bound[1] <= pixel[1] <= upper_bound[1] and lower_bound[2] <= pixel[2] <= upper_bound[2]:
                frame[i][j] = replacement[i][j]
    return frame

In [36]:
time_spent = process_video(video_path, replacement_path, img_size, lower_bound, upper_bound, chroma_jit)
print(round(time_spent, 2))

21.44
